In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

# 비용함수 구현하기 (로우레벨)

In [2]:
z = torch.FloatTensor([1, 2, 3])

In [3]:
hypothesis = F.softmax(z, dim=0)
# 3개의 원소 값이 0과 1사이 값을 가지는 벡터로 변환된다.
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
hypothesis.sum() # 3개의 원소의 총 합은 1이다.

tensor(1.)

### 비용 함수 직접 구현해보기

In [5]:
# 비용함수 직접 구현해보기
z = torch.rand(3, 5, requires_grad=True) # 임의로 3*5 크기의 텐서 구현

In [6]:
hypothesis = F.softmax(z, dim=1)
# 각 샘플에 대해 소프트맥스 함수를 적용해야 하므로
# 두번째 차원에 대해 소프트맥스 함수를 적용시킨다. (dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


- 각 행의 원소들의 합이 1이 되는 텐서로 변환하였다.
- 위 텐서는 3개의 샘플에 대해 5개의 클래스 중 어떤 클래스가 정답인지 예측한 결과이다.

In [7]:
# 각 샘플에 대해 임의의 레이블 만들기
y = torch.randint(5, (3,)).long() # 0~5 사이의 값으로 채워진 3차원 텐서
print(y)

tensor([0, 2, 1])


### 원-핫 인코딩 수행

In [8]:
# One-Hot Encoding 수행
y_one_hot = torch.zeros_like(hypothesis) # 0으로 채워진 3*5 텐서 생성해 y_one_hot에 저장
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

- 두번째 줄에서 scatter의 첫번째 인자로 dim=1에 대해 수행하라고 명령한다.
- 그리고 두번째 인자의 y.unsqueeze(1)을 통해 (3,)의 크기를 가졌던 y 텐서가 (3*1)텐서가 된다.
    ```py
    print(y.unsqueeze(1))
    ```
    ```
    tensor(
        [[0],
         [2],
         [1]])
    ```
- 세번째 인자에 숫자 1을 넣어 y_unsqeeze(1)이 알려주는 위치에 숫자 1을 넣도록 한다. (연산 뒤에 _를 붙이면 In-place Operation(덮어쓰기 연산)이다.)

In [9]:
# y_one_hot의 최종결과
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


![수식](../img/SR19.png)
- 위의 소프트맥스 회귀의 비용 함수는 마이너스 부호를 뒤로 빼서 아래와 같이 만들 수 있다.
![수식](../img/SR24.png)
- ![수식](../img/SR25.png)는 sum(dim=1)으로 구현하고, ![수식](../img/SR26.png)는 mean()으로 구현한다.

In [10]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


# 비용함수 구현하기 (하이-레벨)

## F.softmax() + torch.log() = F.log_softmax()

In [11]:
# Low Level
torch.log(F.softmax(z, dim=1)) # 소프트맥스 함수 출력값에 로그를 씌운다.

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [12]:
# High Level
F.log_softmax(z, dim=1) # PyTorch에서 위 두개의 함수를 결합한 F.log_softmax() 제공

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

## F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [13]:
# Low Level로 구현한 비용함수
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [14]:
# F.log_softmax() 사용
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
# High Level
F.nll_loss(F.log_softmax(z, dim=1), y)

- F.nll_loss()를 사용할 때는 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용한다.
- nll : Negative Log Likelihood
- nll_loss : F.log_softmax()를 수행한 후 남은 수식들을 수행한다.

In [15]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

- F.cross_entropy : F.log_softmax()와 F.nll_loss()를 포함한다. 비용 함수에 소프트맥스 함수까지 포함하고 있다.